## Retrievers
Now that we understand the indexing pipeline, we can utilize our vector database to retrieve relevant documents for a given query.

LangChain provides a uniform interface for interacting with different types of retrieval systems. The LangChain retriever interface is straightforward:

Input: A query (string) Output: A list of documents (standardized LangChain Document objects)

https://python.langchain.com/docs/concepts/retrievers/

In [2]:
%pip install -qU langchain langchain_community wikipedia

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()
docs = retriever.invoke("Indian Independence")

docs[0].page_content[:200]

c:\Users\kirth\OneDrive\Desktop\AI_Software_Engineer_Kirthika\Github_Projects_for_AI\RAG-LangChain\venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\kirth\OneDrive\Desktop\AI_Software_Engineer_Kirthika\Github_Projects_for_AI\RAG-LangChain\venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'The Indian independence movement was a series of political efforts from the middle of the nineteenth century to 1947, that took place in the Indian subcontinent with the aim of ending British colonial'

In [6]:
%pip install -qU langchain-huggingface

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

loader = WebBaseLoader("https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_documents(chunks, embeddings)


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\kirth\OneDrive\Desktop\AI_Software_Engineer_Kirthika\Github_Projects_for_AI\RAG-LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\kirth\OneDrive\Desktop\AI_Software_Engineer_Kirthika\Github_Projects_for_AI\RAG-LangChain\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kirth\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment

In [7]:
# Using a Vector store as a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # Get top 2 results
docs = retriever.invoke("How long can senators serve?") # Ask a question from the constitution

def print_docs(docs):
    for doc in docs:
        print(doc.page_content[:500])
        print("-"*100+"\n")

print_docs(docs)

Qualification for office. No religious              6    .......       3
 test shall ever be required as a.........
Qualifications of electors of members of            1          2       1
 the House of Representatives shall be the
 same as electors for the most numerous
 branch of the State legislature..........
Qualifications of electors of Senators             17          1  ......
 shall be the same as electors of the most
 numerous branch of the State legislature.
 [Amendments].............
----------------------------------------------------------------------------------------------------

Article [XVII.]

    The Senate of the United States shall be composed of two 
Senators from each State, elected by the people thereof, for 
six years; and each Senator shall have one vote. The electors 
in each State shall have the qualifications requisite for 
electors of the most numerous branch of the State legislatures.
    When vacancies happen in the representation of any State in 
the S